In [1]:
import csv
import pprint
import pandas as pd
import pywikibot
import re
from string import punctuation
import urllib.request, json
import urllib.parse

from caphi2ipa import caphipp2ipa

# To do

* Singular nouns entries
    * Add plurals
* Loop over lemma (instead of PoS) and create multi-term entries
    * 87 + 107 https://en.wiktionary.org/w/index.php?title=User%3AA455bcd9%2FMaknuune%2F%D8%A3%D8%AE%D8%B0&diff=prev&oldid=70822528&diffmode=source
    * 7985 + 7988 https://en.wiktionary.org/w/index.php?title=User%3AA455bcd9%2FMaknuune%2F%D8%AE%D8%B1%D9%82%D8%A9&diff=prev&oldid=70822545&diffmode=source
* Plural nouns entries
* Adjectives
* Active participles (NOUN_ACT:MS)
* Passive participles (NOUN_PASS)
* INTERJ
* ADV
* Check entries already in Wiktionary
* NOUN:PHRASE
* Verbs
* VERB:PHRASE
* Issues:
    * شرعية: WARNING: API warning (edit): The value passed for "text" contains invalid or non-normalized data. Textual data should be valid, NFC-normalized Unicode without C0 control characters other than HT (\t), LF (\n), and CR (\r).

# Read TSV

In [7]:
maknuune_file = "maknuune-v1.0.1.tsv"

singular_nouns = ['NOUN:MS', 'NOUN:FS']

df = pd.read_csv(maknuune_file,sep='\t')

# Stats by PoS

In [8]:
df_count = df.groupby(['ANALYSIS'])['ANALYSIS'].count()
df_count.loc[lambda x : x > 50].sort_values(ascending=False)

ANALYSIS
VERB:I             6345
VERB:C             6329
VERB:P             6191
NOUN:MS            4183
NOUN:FS            3050
ADJ:MS             2388
NOUN:PHRASE        2343
NOUN:P             1894
NOUN_ACT:MS         759
VERB:PHRASE         639
NOUN_PASS           305
ADJ:P               288
ADJ:PHRASE          249
NOUN_ACT:PHRASE     117
ADJ/NOUN            108
INTERJ              106
ADJ:FS               78
ADJ_COMP             77
ADV                  71
NOUN_PROP            65
Name: ANALYSIS, dtype: int64

# First: singular nouns deriving from a root and without notes 

In [9]:
# 1355 entries with notes => deal with them later
# df.loc[df['NOTES'].notnull()].count()

# 897 entries with NTWS roots => deal with them later
# df.loc[df['ROOT'] == 'NTWS'].count()

# df['LEMMA_BW'] == df['FORM_BW'] to exclude weird cases for nouns

df_singular_nouns = df.loc[(df['ROOT'] != 'NTWS') & (df['ANALYSIS'].isin(singular_nouns)) & (df['LEMMA_BW'] == df['FORM_BW']) & (df['NOTES'].isnull())]

df_singular_nouns

,ID,ROOT,ROOT_NTWS,ROOT_1,LEMMA,LEMMA_SEARCH,FORM,LEMMA_BW,FORM_BW,CAPHI++,ANALYSIS,GLOSS,GLOSS_MSA,EXAMPLE_USAGE,NOTES,SOURCE,ANNOTATOR,Unnamed: 17,Unnamed: 18,Unnamed: 19
0,1,ء.ب.د,NaN,ء,أَبَد,أبد,أَبَد,>abad,>abad,2 a b a d,NOUN:MS,eternity;forever,NaN,NaN,NaN,NaN,شهد دعباس,NaN,NaN,NaN
1,2,ء.ب.ر,NaN,ء,إِبْرِة,إبرة,إِبْرِة,<iborip,<iborip,2 i b r e,NOUN:FS,needle;injection,إِبْرَة#حُقْنَة,في إِبْرِة وقعت تحت الكنب دير بالك.#أخذت ابرة ...,NaN,NaN,شهد دعباس,NaN,NaN,NaN
7,8,ء.ب.ط,NaN,ء,أَبَاط,أباط,أَبَاط,>abaAT,>abaAT,2 a b aa t.,NOUN:MS,armpit,إِبْط,ولك ليش أباطك أسود هيك؟,NaN,NaN,شهد دعباس,NaN,NaN,NaN
8,9,ء.ب.ط,NaN,ء,بَاط,باط,بَاط,baAT,baAT,b aa t.,NOUN:MS,armpit,إِبْط,NaN,NaN,NaN,شهد دعباس,NaN,NaN,NaN
10,11,ء.ب.و,NaN,ء,أَب,أب,أَب,>ab,>ab,2 a b,NOUN:MS,father,أب,الله يرحمه أبوه بقى زلمة مليح وبينشد فيه الظهر,NaN,NaN,شهد دعباس,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36039,36263,ي.م.ن,NaN,ي,يَمِين,يمين,يَمِين,yamiyn,yamiyn,y a m ii n,NOUN:MS,right,يمين,روتش لك شوي لليمين,NaN,NaN,شهد دعباس,NaN,NaN,NaN
36040,36264,ي.م.ن,NaN,ي,يَمِين,يمين,يَمِين,yamiyn,yamiyn,y a m ii n,NOUN:MS,vow;oath,NaN,احلف يَمِين إِنه مالكاش دخَّل!,NaN,NaN,شهد دعباس,NaN,NaN,NaN
36044,36268,ي.م.ن,NaN,ي,يُمُن,يمن,يُمُن,yumun,yumun,y u m u n,NOUN:MS,blessing,نِعْمَة,ألف مبارك الشهر وكل عام وانتو بخير وان شاء الل...,NaN,NaN,شهد دعباس,NaN,NaN,NaN
36056,36280,ي.و.م,NaN,ي,يَومِيِّة,يومية,يَومِيِّة,yawmiy~ip,yawmiy~ip,y oo m i y y e,NOUN:FS,daily_payment,دفع بنظام المياومة,NaN,NaN,NaN,شهد دعباس,NaN,NaN,NaN


# Wiktionary format

In [18]:
beginning_file = """'''User:A455bcd9/Maknuune/{entry}'''
{{{{-start-}}}}"""

template = """==South Levantine Arabic==
{{{{bulk import|{{{{R:ajp:Maknuune|id={id}}}}}}}}}
{{{{ajp-root|{root}}}}}

===Pronunciation===

* {{{{ajp-IPA|{ipa}}}}}

===Noun===
{{{{ajp-noun|head={head}|g={gender}|tr={transliteration}{plural}}}}}

# {glosses}{example}

===References===
* {{{{R:ajp:Maknuune|id={id}}}}}"""

end_file="{{{{-stop-}}}}"

"""
Plural:

===Noun===
{{ajp-noun|head={head}|g=p|tr={transliteration}}}
# {{plural of|ajp|{singular}|tr={singulartransliteration}|t={singulargloss}}}
# [[definition]]

In singular entry:
If the noun has no plural: |pl=-
If there are multiple plural forms: |pl= and |pl2=, |pl3= // |pltr= and |pl2tr=, |pl3tr=

"""

'\nPlural:\n\n===Noun===\n{{ajp-noun|head={head}|g=p|tr={transliteration}}}\n# {{plural of|ajp|{singular}|tr={singulartransliteration}|t={singulargloss}}}\n# [[definition]]\n\nIn singular entry:\nIf the noun has no plural: |pl=-\nIf there are multiple plural forms: |pl= and |pl2=, |pl3= // |pltr= and |pl2tr=, |pl3tr=\n\n'

# Select 10 random entries

In [11]:
# df_singular_nouns = df_singular_nouns.sample(n=1000)

# CAPHI++ to Wikt

In [12]:
for index, row in df_singular_nouns.iterrows():
    caphipp = row['CAPHI++'].strip()
    ipa = caphipp2ipa(caphipp, mode='wikt')
    # print(row['LEMMA'], '=>', caphipp, '=>', ipa)
    
# test m a w J uu d => mawjūd (موجود)
# maw(dʒ)uːd // مَوْ ُجود
print(caphipp2ipa('m a w J uu d', mode='wikt'))
# 2 i sh t aa Q => ištāʔ (اشتاق)
# ʔiʃtaː(q)
print(caphipp2ipa('2 i sh t aa Q', mode='wikt'))

mawjūd
ʔištāʔ


# Test entries

In [13]:
test_entries = [
    'أب',
    'أباط',
    'أجار',
    'أجرة',
    'أخذ',
    'ألفة',
    'إيجار',
    'باط',
    'بلع',
    'تأثير',
    'تأجير',
    'تأجيل',
    'تجارة',
    'خرقة',
    'زهاب',
    'عنصرية',
    'غواصة',
    'قذلة',
    'كماجة',
    'لقمة',
    'مهاجر',
    'مواخذة',
    'شرعية',
    'بيئة',
    'كيلو',
    'مفاد',
    'خضرة',
    'نظرة',
    'فناء',
    'إفشاء',
    'عجلة',
    'صحصحة',
    'واعظ',
]

# Save entries in Wiktionary

## Clean glosses

In [14]:
def clean_gloss(gloss):
    # Clean and link glosses
    clean_gloss = gloss
    
    replacements = [
        (u"\u2018", "'"),
        (u"\u2019", "'"),
        ("_", " "),
        (r' +', ' ',),
        ('sb/sth', 'somebody or something'),
        ("(?i) sth", " something"),
        ("(?i) sb", " somebody"),
        (r"(?i)^sb ", "somebody "),
        (r"(?i)^sth ", "something "),
        ("(?i) some one ", " someone "),
        (r"(?i)^a type of ", ""),
        (r"(?i)^see phrase(s)?", ""),
        (r"(?i)^a ", ""),
        (r"(?i)^an ", ""),
        (r"(?i)^The ", ""),
        (r"(?i)^It is what ", "what "),
        (r"(?i)^It is (a|the) ", ""),
        (r"(?i)^It is like (a|the) ", ""),
        (' that is made of ', ' made of '),
        (' that is made from ', ' made from '),
        (' a n ', ' an ')
    ]
    
    for replacement in replacements:
        clean_gloss = re.sub(replacement[0], replacement[1], clean_gloss)
        
    clean_gloss = clean_gloss.strip(' .')
    
    return clean_gloss

def link_gloss(gloss):
    # clean gloss
    linked_gloss = clean_gloss(gloss)
    
    # if single word, link it
    if len(linked_gloss.split(' ')) == 1 and linked_gloss != '':
        linked_gloss = u"[[{g}]]".format(g=linked_gloss)
    elif len(linked_gloss.split(' ')) in [2, 3]:
        # Check whether English n-grams (glosses) are already in the Wiktionary and link them
        api_url = u"https://en.wiktionary.org/w/api.php?action=query&titles={entry}&format=json".format(entry=linked_gloss).replace(' ', '%20')
        with urllib.request.urlopen(api_url) as url:
            data = json.load(url)
            # if the entry exists, link it
            if not data.get('query').get('pages').get('-1'):
                linked_gloss = u"[[{g}]]".format(g=linked_gloss)
    
    return linked_gloss

In [15]:
link_gloss('narrow path')
link_gloss('take down')

'[[take down]]'

In [17]:
def test_clean_gloss():
    to_test = [
        ('a_type_of_jar_for_keeping_water_cold', 'jar for keeping water cold'),
        ('see_phrase', ''),
        ('ceramic work.', 'ceramic work'),
        ('sb_who_eats_food_off_the_ground', 'somebody who eats food off the ground'),
        ('a small ball', 'small ball'),
        ('a single person', 'single person'),
        ('a narrow path', 'narrow path'),
        ('a fat person', 'fat person'),
        ('sb/sth', 'somebody or something'),
        ('a difficult  situation', 'difficult situation'),
        ('the grated food', 'grated food'),
        ('the beautiful woman', 'beautiful woman'),
        ('a_stone_inside_a_hole', 'stone inside a hole'),
        ('an individual.', 'individual'),
        ('see phrases', ''),
        ("the_state_of_holding_something_over_sb's_head", "state of holding something over somebody's head"),
        ('a_vessel_that_is_usually_of_brass_for_kohl.', 'vessel that is usually of brass for kohl'),
        ('It_is_what_is_placed_on_the_back_of_the_walking_animal_for_the_purpose_of_riding_on_it', 'what is placed on the back of the walking animal for the purpose of riding on it'),
        ('It_is_the_ritual_animal_sacrifice_of_a_livestock_animal_in_the_first_three_days_of_the_funeral_that_is_believed_to_give_solace_to_the_deceased_person', 'ritual animal sacrifice of a livestock animal in the first three days of the funeral that is believed to give solace to the deceased person'),
        ('A_thin_layer_of_freezing_water_in_the_morning,_due_to_lower_temperatures.', 'thin layer of freezing water in the morning, due to lower temperatures'),
        ('It_is_a_traditional_type_of_dessert_that_is_made_of_dough,_water_and_molasses._It_is_usually_eaten_with_dried_figs.', 'traditional type of dessert made of dough, water and molasses. It is usually eaten with dried figs'),
        ('sth_similar_to_a_container_that_is_made_by_sewing_several_"7_u_s._u_r"_together._The_farmers_usually_keep_wheat_in_it.', 'something similar to a container that is made by sewing several "7 u s. u r" together. The farmers usually keep wheat in it'),
        ('the_person_whose_job_is_to_tell_stories_to_the_people', 'person whose job is to tell stories to the people'),
        ('It_is_like_a_basket_made_of_rubber,_or_of_wicker_and_fiber,_and has two loops that are used as handles for carrying it. People use it to carry multiple things.', 'basket made of rubber, or of wicker and fiber, and has two loops that are used as handles for carrying it. People use it to carry multiple things'),
        ('traditional Levantine dish that is made of chickpeas', 'traditional Levantine dish made of chickpeas'),
        ('small container that is made from skinned sheepskin', 'small container made from skinned sheepskin'),
        ('speak_in_a_n_incomprehensible_way', 'speak in an incomprehensible way'),
    ]
    for unit_test in to_test:
        if clean_gloss(unit_test[0]) != unit_test[1]:
            raise ValueError('Unit test failed:', clean_gloss(unit_test[0]), unit_test[1])
            
    return 'clean_gloss(): OK'

test_clean_gloss()

'clean_gloss(): OK'

In [29]:
site = pywikibot.Site()
i = 0

for index, row in df_singular_nouns.iterrows():
    # remove _[auto] from entries generated automatically from other entries
    clean_gloss_row = row['GLOSS'].replace('_[auto]', '').strip()
    # filter out more complex glosses
    entry_values = {}
    entry_values['id'] = row['ID']

    entry_values['entry'] = row['LEMMA_SEARCH'].strip()
    entry_values['root'] = row['ROOT'].strip().replace('.', ' ')
    entry_values['head'] = row['FORM'].strip()
    entry_values['gender'] = 'f' if row['ANALYSIS'] == 'NOUN:FS' else 'm'

    caphipp = row['CAPHI++'].strip()
    entry_values['transliteration'] = caphipp2ipa(caphipp, mode='wikt')
    entry_values['ipa'] = entry_values['transliteration']
    
    plurals = df.loc[(df['LEMMA'] == row['LEMMA']) & (df['ANALYSIS'] == 'NOUN:P')]
    
    n_plurals = len(plurals.index)
    if n_plurals == 0:
        # No plurals in the database for this singular noun
        entry_values['plural'] = ''
    else:
        # At least one plural in the database for this singular noun
        plurals_list = []
        j = 1
        for index_pl, row_pl in plurals.iterrows():
            plurals_list.append(u"|pl{i}={pl}|pl{i}tr={pltr}".format(
                i= str(j) if j > 1 else '',
                pl=row_pl['FORM'].strip(),
                pltr=caphipp2ipa(row_pl['CAPHI++'].strip(), mode='wikt')))
            j += 1

        entry_values['plural'] = ''.join(plurals_list)        

    entry_values['example'] = ''
    if not pd.isna(row['EXAMPLE_USAGE']):
        example = row['EXAMPLE_USAGE'].strip()
        # Fill example if not null
        if example != '':
            entry_values['example'] = '\n#: {{{{ux|ajp|{example}}}}}'.format(example=example)

    # Separate glosses
    glosses = set(clean_gloss_row.split(';'))
    
    # Do not add empty glosses
    if len(glosses) == 1:
        if link_gloss(list(glosses)[0]) == '':
            continue
        
    # clean glosses and link simple ones
    entry_values['glosses'] = ', '.join([link_gloss(x) for x in glosses])

    new_entry = template.format(**entry_values)

    to_save = entry_values['entry'] in test_entries
    
    if to_save:
        page_name = u"User:A455bcd9/Maknuune/{entry}".format(entry=entry_values['entry'])
        page = pywikibot.Page(site, page_name)

        page.text = new_entry
        print(new_entry)
        page.save(u"Entries with plural")
        # i += 1    
            
    if i > 20:
        break

print('end')

==South Levantine Arabic==
{{bulk import|{{R:ajp:Maknuune|id=8}}}}
{{ajp-root|ء ب ط}}

===Pronunciation===

* {{ajp-IPA|ʔabāṭ}}

===Noun===
{{ajp-noun|head=أَبَاط|g=m|tr=ʔabāṭ}}

# [[armpit]]
#: {{ux|ajp|ولك ليش أباطك أسود هيك؟}}

===References===
* {{R:ajp:Maknuune|id=8}}


Page [[User:A455bcd9/Maknuune/أباط]] saved


==South Levantine Arabic==
{{bulk import|{{R:ajp:Maknuune|id=9}}}}
{{ajp-root|ء ب ط}}

===Pronunciation===

* {{ajp-IPA|bāṭ}}

===Noun===
{{ajp-noun|head=بَاط|g=m|tr=bāṭ}}

# [[armpit]]

===References===
* {{R:ajp:Maknuune|id=9}}


Sleeping for 9.1 seconds, 2023-01-13 14:55:19
Page [[User:A455bcd9/Maknuune/باط]] saved


==South Levantine Arabic==
{{bulk import|{{R:ajp:Maknuune|id=11}}}}
{{ajp-root|ء ب و}}

===Pronunciation===

* {{ajp-IPA|ʔab}}

===Noun===
{{ajp-noun|head=أَب|g=m|tr=ʔab|pl=إِبْوِة|pltr=ʔibwe|pl2=آبَاء|pl2tr=ʔābāʔ|pl3=أَبَّهَات|pl3tr=ʔabbahāt}}

# [[father]]
#: {{ux|ajp|الله يرحمه أبوه بقى زلمة مليح وبينشد فيه الظهر}}

===References===
* {{R:ajp:Maknuune|id=11}}


...

==South Levantine Arabic==
{{bulk import|{{R:ajp:Maknuune|id=34062}}}}
{{ajp-root|ه ج ر}}

===Pronunciation===

* {{ajp-IPA|muhājir}}

===Noun===
{{ajp-noun|head=مُهَاجِر|g=m|tr=muhājir}}

# [[immigrant]]

===References===
* {{R:ajp:Maknuune|id=34062}}


Page [[User:A455bcd9/Maknuune/مهاجر]] saved


==South Levantine Arabic==
{{bulk import|{{R:ajp:Maknuune|id=35743}}}}
{{ajp-root|و ع ظ}}

===Pronunciation===

* {{ajp-IPA|wāʕiẓ}}

===Noun===
{{ajp-noun|head=وَاعِظ|g=m|tr=wāʕiẓ}}

# [[sheikh]], [[priest]]
#: {{ux|ajp|جابوا واعِظة على عزا عمتي الله يرحمها}}

===References===
* {{R:ajp:Maknuune|id=35743}}


Page [[User:A455bcd9/Maknuune/واعظ]] saved


end


In [ ]:
# add plurals


In [42]:
"""
To check: 

New line:
2955 state of being filled to the max or stuffed
6194 traditional dish that is made of unripe grapes and lentils
35498 being handsome
"""

'\nissues:\n\nsee_phrases\nsee_phrase:\n33175\n33177 \n33179 \n735\n968\n1212\n2202\n13973\n\nNew line:\n2955 state of being filled to the max or stuffed\n6194 traditional dish that is made of unripe grapes and lentils\n35498 being handsome\n'